# Model Risk Pipeline

This notebook aims at demonstrating how one might employ the default pipeline for ML tasks. Hyperparameter optimization relies on `hyperopt` library, we expect to add `optuna` support in the future.

__Inputs:__

1. Dataset

__Ouputs:__

1. Optimal pipeline structure, as obtained by `hyperopt`
2. Optimal model

## 1. Technicals

In [1]:
import time
import json
import numpy as np
import pandas as pd
import sklearn
from hyperopt import hp
from collections import OrderedDict
import matplotlib.pyplot as plt

from modules.utils import gini_score
from modules.defaults import get_default_modules
from modules.pipeline import PipeHPOpt

from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

In [2]:
seed = 42

## 2. Import Dataset

In [3]:
X_train = pd.read_parquet('../datasets/01_german/samples/X_train.parquet')
y_train = pd.read_parquet('../datasets/01_german/samples/y_train.parquet').target

X_test  = pd.read_parquet('../datasets/01_german/samples/X_test.parquet')
y_test  = pd.read_parquet('../datasets/01_german/samples/y_test.parquet').target

In [4]:
with open('../datasets/01_german/factors.json') as json_file:
    factors_dict = json.load(json_file)

## 3. Define Modules

All the modules that might be part of the pipeline should be defined below (or import them):

In [5]:
# get default modules
pipe_modules = get_default_modules()

## 4. Define Pipeline

### 4.1. Структура Pipeline

Определим структуру самого пайплайна. Словесное описание:
    
1. Энкодинг категориальных переменных:
    + OneHotEncoder
    + WoE
3. Feature Engineering:
    + PCA
    + Kernel PCA
    + Isomap
    + UMAP
    + Combine with Reference (feature multiplication)
    + _passthrough_
4. Feature Selection:
    + RecursiveFeatureAddition
    + SequentialFeatureSelector
    + SmartCorrelatedSelection
    + _passthrough_
4. Resampling:
    + Randomised Undersampling (RUS)
    + Randomised Oversampling  (ROS)
    + Synthetic Minority Oversampling Technique (SMOTE)
    + Adaptive Synthetic (ADASYN)
    + _passthrough_
5. LightGBM

А так это будет выражаться в коде:

In [6]:
pipe_params = OrderedDict()

pipe_params['cat_encoding'] = hp.choice('cat_encoding', ['OneHot', 'WoE'])
pipe_params['missing_vals'] = hp.choice('missing_vals', ['skip', 'MeanImp', 'MedImp']) # , 'ModeImp', 'RandomImp', 'KNNImp', 'IterImp'
pipe_params['imbalance']    = hp.choice('imbalance',    ['skip', 'RUS', 'ROS', 'SMOTE', 'ADASYN'])
pipe_params['feat_eng']     = hp.choice('feat_eng',     ['skip', 'PCA', 'kPCA', 'Isomap', 'UMAP']) # , 'CombWRef' excluded, since it takes too much time
pipe_params['feat_sel']     = hp.choice('feat_sel',     ['skip', 'SeqFearSel', 'RecFeatAdd', 'SmartSel']) # 'SelShuffl' is omitted, since it might drop all Xs
pipe_params['lgbm']         = 'lgbm'

pipe_params
# hp.choice() -- unstable, low quality (SHAP, IV, feat_importance, ...) 

OrderedDict([('cat_encoding', <hyperopt.pyll.base.Apply at 0x23df634c2c8>),
             ('missing_vals', <hyperopt.pyll.base.Apply at 0x23df634c7c8>),
             ('imbalance', <hyperopt.pyll.base.Apply at 0x23df634ce48>),
             ('feat_eng', <hyperopt.pyll.base.Apply at 0x23df634b448>),
             ('feat_sel', <hyperopt.pyll.base.Apply at 0x23df634b948>),
             ('lgbm', 'lgbm')])

Заметим, что 'skip' позволяет игнорировать соответствующий шаг в пайплайне. Названия типа `"onehot"` должны совпадать с названиями в словаре `modules`, который мы определили на Шаге 3.

### 4.2. Гиперпараметры модулей

В следующий словарь добавляем гиперпараметры каждого из модулей, которые мы хотим оптимизировать. Названия строятся следующи образом:

`<Название модуля>__<название параметра>`

Например, чтобы задать параметр `num_leaves` модуля lgbm, трубуется добваить значение с ключем `lgbm__num_leaves`.

In [7]:
set_params = {
    # OneHotEncoder does not need hyperparams
    # RecFeatAdd might be redefined to receive a correct estimator
    # PCA
    'DimRed__PCA__n_components':      hp.choice('PCA__n_components',      np.arange(2, 11)),
    'DimRed__PCA__whiten':            hp.choice('PCA__whiten',            [True, False]),
    'DimRed__PCA__svd_solver':        hp.choice('PCA__svd_solver',        ['full', 'arpack', 'auto', 'randomized']),
    
    # kPCA
    'DimRed__kPCA__n_components':     hp.choice('kPCA__n_components',     np.arange(5, 11)),
    'DimRed__kPCA__kernel':           hp.choice('kPCA__kernel',           ['linear', 'poly', 'rbf', 'sigmoid', 'cosine', 'precomputed']),
    
    # Isomap
    'DimRed__Isomap__n_neighbors':    hp.choice('Isomap__n_neighbors',    np.arange(2, 11)),
    'DimRed__Isomap__n_components':   hp.choice('Isomap__n_components',   np.arange(2, 5)),
    'DimRed__Isomap__path_method':    hp.choice('Isomap__path_method',    ['auto', 'FW', 'D']),
    
    # UMAP
    'DimRed__UMAP__n_neighbors':      hp.choice('UMAP__n_neighbors',      np.arange(2, 11)),
    'DimRed__UMAP__n_components':     hp.choice('UMAP__n_components',     np.arange(2, 11)),
    'DimRed__UMAP__min_dist':         hp.choice('UMAP__min_dist',         np.arange(0.05, 1, 0.05)),
    
    # LightGBM
    'lgbm__learning_rate':            hp.choice('lgbm__learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'lgbm__num_leaves':               hp.choice('lgbm__num_leaves',       np.arange(5, 16, 1, dtype=int)),
    'lgbm__reg_alpha':                hp.choice('lgbm__reg_alpha',        np.arange(0, 16, 1, dtype=int)),
    'lgbm__reg_lambda':               hp.choice('lgbm__reg_lambda',       np.arange(0, 16, 1, dtype=int)),
    'lgbm__n_estimators':             100
}

Чтобы параметры можно было оптимизировать, модули должны иметь метод `.set_params()`.

### 4.3. Подстановка оптимизационной задачи

In [8]:
# technicals — define minimization task
pipe_para = dict()
pipe_para['pipe_params']    = pipe_params
pipe_para['set_params']     = set_params
pipe_para['loss_func']      = lambda y, pred: -sklearn.metrics.roc_auc_score(y, pred)
# pipe_para['loss_func']      = lambda y, pred: -sklearn.metrics.log_loss(y, pred)

In [9]:
%%capture --no-stdout
hpoptimizer = PipeHPOpt(X_train, y_train, modules=pipe_modules, mode='kfold', n_folds = 4, seed=seed)
lgb_opt, trials = hpoptimizer.process(space=pipe_para, trials=Trials(), algo=tpe.suggest, max_evals=10)

100%|███████████████████████████████████████████████| 10/10 [03:26<00:00, 20.64s/trial, best loss: -0.7591530162432008]


## 5. Results Analysis

In [12]:
hpoptimizer.get_best_params()
best_mdl = hpoptimizer.get_best_model()

#### Ginit: Train vs. Test

In [13]:
gini_score(y_train, best_mdl.predict_proba(X_train)[:, 1])
gini_score(y_test, best_mdl.predict_proba(X_test)[:, 1])

Gini: 0.8682888085570706
Gini: 0.6509195816804905


0.6509195816804905

#### HyperOpt Convergence

<Figure size 432x288 with 0 Axes>

In [14]:
plt.plot(np.array([r['loss'] for r in trials.results]))
plt.title('Hyperopt: loss function dynamics')
plt.xlabel('Epoch')
plt.ylabel('Average loss (-AUC) on k cross-validation samples')
plt.show()
plt.savefig('loss_dynamics_hp_1000_tpe_suggest.png')